In [19]:
import ants
import glob
import os
import shutil

In [2]:
ground_truth_filenames = sorted(glob.glob("./preprocessed/*/*/*NB*.nii.gz"))
print(len(ground_truth_filenames))

77


In [3]:
out_dir = "./aligned"
os.makedirs(out_dir, exist_ok=True)

In [20]:
for label_filename in ground_truth_filenames:
    subject, visit = label_filename.split("/")[-3:-1]
    filenames = sorted(glob.glob(os.path.join("train", subject, visit, f"*.nii*")))
    n_features = len(filenames) 
    feature_modalities = ["_".join(fn.split("_")[3:-1]) for fn in filenames]
    t1_filename = filenames[feature_modalities.index("T1_gd")]
    assert os.path.exists(t1_filename)
    fixed = ants.image_read(t1_filename)
    print("read image")
    for filename in filenames:
        if filename not in (t1_filename, label_filename):
            out_filename = os.path.abspath(filename).replace("train", "aligned")
            if ".nii" in out_filename and ".nii.gz" not in out_filename:
                out_filename = out_filename.replace(".nii", ".nii.gz")
            if os.path.exists(out_filename):
                continue
            moving = ants.image_read(filename)
            print("read moving")
            result = ants.registration(moving=moving, fixed=fixed, type_of_transform="QuickRigid")
            print("finished registration")

            os.makedirs(os.path.dirname(out_filename), exist_ok=True)
            print("writing", out_filename)
            assert out_filename != filename
            ants.image_write(result['warpedmovout'], out_filename)
            
    t1_out = os.path.abspath(t1_filename).replace("train", "aligned")
    print(t1_out)
    assert t1_out != t1_filename
    shutil.copy(t1_filename, t1_out)
    label_out = os.path.abspath(label_filename).replace("preprocessed", "aligned")
    print(label_out)
    assert label_out != label_filename
    shutil.copy(label_filename, label_out)


read image
/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_06/20180628/PT_06_T1_gd_20180628.nii
/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_06/20180628/PT_06_NB_20180628.nii.gz
read image
read moving
finished registration
writing /lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_07/20180801/PT_07_DWI_b0_20180801.nii.gz
read moving
finished registration
writing /lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_07/20180801/PT_07_DWI_b100_20180801.nii.gz
read moving
finished registration
writing /lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_07/20180801/PT_07_T2_20180801.nii.gz
read moving
finished registration
writing /lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_07/20180801/PT_7_NB_20180801.gz.nii.gz
/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_07/201808

In [21]:
out_filename

'/lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_92/20181026/PT_92_T2_20181026.nii.gz'

In [16]:
os.remove(out_filename)

In [12]:
out_filename = os.path.abspath(filename).replace("train", "aligned").replace(".nii", ".nii.gz")
os.makedirs(os.path.dirname(out_filename), exist_ok=True)
print("writing", out_filename)
ants.image_write(result['warpedmovout'], out_filename)

writing /lustre/work/aizenberg/dgellis/MICCAI23/code/3DUnetCNN/examples/sppin/aligned/PT_06/20180628/PT_06_DWI_b0_20180628.nii.gz


In [2]:
ants.registration(type_of_transform="QuickRigid")

Signature:
ants.registration(
    fixed,
    moving,
    type_of_transform='SyN',
    initial_transform=None,
    outprefix='',
    mask=None,
    moving_mask=None,
    mask_all_stages=False,
    grad_step=0.2,
    flow_sigma=3,
    total_sigma=0,
    aff_metric='mattes',
    aff_sampling=32,
    aff_random_sampling_rate=0.2,
    syn_metric='mattes',
    syn_sampling=32,
    reg_iterations=(40, 20, 0),
    aff_iterations=(2100, 1200, 1200, 10),
    aff_shrink_factors=(6, 4, 2, 1),
    aff_smoothing_sigmas=(3, 2, 1, 0),
    write_composite_transform=False,
    random_seed=None,
    verbose=False,
    multivariate_extras=None,
    restrict_transformation=None,
    smoothing_in_mm=False,
    **kwargs,
)
Docstring:
Register a pair of images either through the full or simplified
interface to the ANTs registration method.

ANTsR function: `antsRegistration`

Arguments
---------
fixed : ANTsImage
    fixed image to which we register the moving image.

moving : ANTsImage
    moving image to be